In [1]:
import os
import SimpleITK as sitk
import glob


#### **数据探索**

In [2]:
# label 是121 个图，image 是176个图
label_dir = '/data/liucd/Dataset/LungSeg/ct-lungseg/nrrd_lung/nrrd_lung'
image_dir = '/data/liucd/Dataset/LungSeg/ct-lung/train'


In [3]:
# 确保所有的label都是有对应的image的
# for image_filename in os.listdir(image_dir):
#     print(image_filename, end=' ')
#     for label_filename in os.listdir(label_dir):
#         if image_filename in label_filename:
#             print(label_filename)
#             break
image_list = [name for name in os.listdir(image_dir)]
label_list = [name.split('_')[0] for name in os.listdir(label_dir)]

In [4]:
merge_list = list(set(image_list) & set(label_list))

In [5]:
len(image_list), len(label_list), len(merge_list)

(177, 111, 110)

#### **数据保存**

In [6]:
# 保存为图像和标签
def dicom2nii(dicom_dir_path):
    reader = sitk.ImageSeriesReader()
    img_name = reader.GetGDCMSeriesFileNames(dicom_dir_path)
    reader.SetFileNames(img_name)
    image = reader.Execute()
    return image

image = dicom2nii(os.path.join(image_dir, 'ID00068637202190879923934/'))
image.GetSize(), image.GetSpacing()  # ((512, 512, 118), (0.78125, 0.78125, 3.051282051282051))

((512, 512, 118), (0.78125, 0.78125, 3.051282051282051))

In [7]:
image = sitk.ReadImage(os.path.join(label_dir, 'ID00102637202206574119190_lung.nrrd'))
image.GetSize(), image.GetSpacing()

((512, 512, 233), (0.7031249999999999, 0.7031249999999999, 1.2500000000000002))

In [11]:
save_dir = '/data/liucd/Dataset/LungSeg'

for idx, file_name in enumerate(merge_list):
    print(file_name)
    image = dicom2nii(os.path.join(image_dir, file_name))
    label = sitk.ReadImage(os.path.join(label_dir,  file_name + '_lung.nrrd'))
    assert image.GetSize() == label.GetSize()
    try:
        assert [(round(idx, 2)) for idx in image.GetSpacing()] == [(round(idx, 2)) for idx in label.GetSpacing()]
    except:
        print([(round(idx, 2)) for idx in image.GetSpacing()], [(round(idx, 2)) for idx in label.GetSpacing()])
    
    label.SetSpacing(image.GetSpacing())
    label.SetOrigin(image.GetOrigin())
    label.SetDirection(image.GetDirection())
        
    sitk.WriteImage(image, os.path.join(save_dir, 'imagesTr', file_name + '.nii.gz'))
    sitk.WriteImage(label, os.path.join(save_dir, 'labelsTr', file_name + '_label.nii.gz'))
    

ID00075637202198610425520
ID00105637202208831864134
ID00351637202289476567312
ID00025637202179541264076
ID00381637202299644114027
[0.88, 0.88, 0.67] [0.88, 0.88, 0.5]
ID00405637202308359492977
ID00343637202287577133798
[0.47, 0.47, 4.74] [0.47, 0.47, 8.0]
ID00222637202259066229764
[0.67, 0.67, 4.93] [0.67, 0.67, 5.0]
ID00210637202257228694086
ID00019637202178323708467
ID00307637202282126172865
ID00048637202185016727717
ID00305637202281772703145
ID00068637202190879923934
[0.78, 0.78, 3.05] [0.78, 0.78, 3.0]
ID00323637202285211956970
ID00184637202242062969203
ID00109637202210454292264
ID00400637202305055099402
ID00111637202210956877205
ID00411637202309374271828
ID00130637202220059448013
ID00299637202280383305867
ID00104637202208063407045
ID00010637202177584971671
ID00073637202198167792918
ID00249637202266730854017
ID00007637202177411956430
ID00192637202245493238298
ID00196637202246668775836
ID00367637202296290303449
ID00197637202246865691526
ID00119637202215426335765
ID000516372021858484

RuntimeError: Exception thrown in SimpleITK ImageSeriesReader_Execute: /tmp/SimpleITK-build/ITK/Modules/IO/GDCM/src/itkGDCMImageIO.cxx:307:
itk::ERROR: itk::ERROR: GDCMImageIO(0x55c20f120fc0): Failed to change to Implicit Transfer Syntax

In [45]:
round(3.1416, 2)

3.14

### **数据EDA**

In [4]:
base_dir = '/data/liucd/Dataset/LungSeg/'
images_dir = os.path.join(base_dir, 'imagesTr')
labels_dir = os.path.join(base_dir, 'labelsTr')
images = sorted(glob.glob(os.path.join(images_dir, '*.nii.gz')))
labels = sorted(glob.glob(os.path.join(labels_dir, '*.nii.gz')))


In [11]:
import numpy as np
spacing_list = []
shape_list = []

for image, label in zip(images, labels):
#     print(image)
#     x = sitk.GetArrayFromImage(sitk.ReadImage(image))
#     y = sitk.GetArrayFromImage(sitk.ReadImage(label))
#     print(np.unique(y))
    shape_list.append(sitk.ReadImage(label).GetSize())
    spacing_list.append(sitk.ReadImage(label).GetSpacing())

In [13]:
shape_arr = np.array(shape_list)
spacing_arr = np.array(spacing_list)

In [18]:
np.median(shape_arr[:, 2])  # z 方向的shape

74.0

In [19]:
np.median(spacing_arr[:, 2])  # z 方向的spacing

5.0

In [20]:
np.median(shape_arr[:, 0]) # x 方向的shape

512.0

In [21]:
np.median(spacing_arr[:, 0])  # z 方向的spacing

0.693359375